<a href="https://colab.research.google.com/github/inoue0426/AVCapturePhotoOutput_test/blob/master/notebooks/scScope.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scScope
!pip install scanpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import scanpy as sc
import scscope as DeepImpute

In [ ]:
!git clone https://github.com/AltschulerWu-Lab/scScope.git

Cloning into 'scScope'...
remote: Enumerating objects: 164, done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 164
Receiving objects: 100% (164/164), 7.69 MiB | 15.75 MiB/s, done.
Resolving deltas: 100% (73/73), done.


In [ ]:
counts_drop = pd.read_csv('scScope/counts_1.csv', header=0, index_col=0)
cellinfo = pd.read_csv('scScope/cellinfo_1.csv', header=0, index_col=0)

In [ ]:
counts_drop.head()

,Gene1,Gene2,Gene3,Gene4,Gene5,Gene6,Gene7,Gene8,Gene9,Gene10,...,Gene991,Gene992,Gene993,Gene994,Gene995,Gene996,Gene997,Gene998,Gene999,Gene1000
Cell1,862,0,0,0,0,1,0,0,0,0,...,0,0,11,0,0,601,0,0,0,60
Cell2,1256,0,0,0,77,0,0,0,0,0,...,0,0,0,0,132,858,23,0,0,0
Cell3,885,0,0,0,0,0,0,0,0,0,...,0,0,0,31,59,522,16,0,0,0
Cell4,546,0,0,0,0,0,10,0,0,0,...,7,0,0,27,0,475,0,0,0,83
Cell5,491,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,14,0


In [ ]:
cellinfo

,Cell,Batch,Group,ExpLibSize
Cell1,Cell1,Batch1,Group1,56535.819348
Cell2,Cell2,Batch1,Group3,86527.830292
Cell3,Cell3,Batch1,Group3,58026.039058
Cell4,Cell4,Batch1,Group4,45344.287050
Cell5,Cell5,Batch1,Group1,44595.097129
...,...,...,...,...
Cell1996,Cell1996,Batch1,Group3,43560.314123
Cell1997,Cell1997,Batch1,Group5,69353.640040
Cell1998,Cell1998,Batch1,Group2,75369.369102
Cell1999,Cell1999,Batch1,Group3,50594.138321


In [ ]:
group = cellinfo.Group
group.head()

Cell1    Group1
Cell2    Group3
Cell3    Group3
Cell4    Group4
Cell5    Group1
Name: Group, dtype: object

In [ ]:
label_ground_truth = []
for g in group:
    g = int(g.split('Group')[1])
    label_ground_truth.append(g)

In [ ]:
label_ground_truth[:10]

[1, 3, 3, 4, 1, 3, 1, 1, 5, 5]

In [ ]:
gene_expression = sc.AnnData(counts_drop.values)
# normalize each cell to have same count number
sc.pp.normalize_per_cell(gene_expression)
gene_expression = gene_expression.X

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  """Entry point for launching an IPython kernel.


In [ ]:
gene_expression

array([[994.4842  ,   0.      ,   0.      , ...,   0.      ,   0.      ,
         69.221634],
       [852.80927 ,   0.      ,   0.      , ...,   0.      ,   0.      ,
          0.      ],
       [922.4004  ,   0.      ,   0.      , ...,   0.      ,   0.      ,
          0.      ],
       ...,
       [642.75604 ,   0.      ,   0.      , ...,   0.      ,   0.      ,
         64.590675],
       [808.89685 ,   0.      ,   0.      , ...,   0.      ,   0.      ,
         64.1416  ],
       [  0.      ,   0.      ,   0.      , ...,   0.      ,   0.      ,
          0.      ]], dtype=float32)

In [ ]:
latent_dim = 50

if gene_expression.shape[0] >= 100000:
    DI_model = DeepImpute.train(
        gene_expression, latent_dim, T=2, batch_size=512, max_epoch=10, num_gpus=4)
else:
    DI_model = DeepImpute.train(
        gene_expression, latent_dim, T=2, batch_size=64, max_epoch=300, num_gpus=4)

AttributeError: ignored